# Passo 1: Preparação dos Dados

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Carregar os dados
summer = pd.read_csv("summer.csv")
wik_1996 = pd.read_csv("wik_1996.csv")
wik_1976 = pd.read_csv("wik_1976.csv")

In [3]:
# Inspecionar os dados
summer.head()

Year    City     Sport Discipline             Athlete Country Gender  \
0  1896  Athens  Aquatics   Swimming       HAJOS, Alfred     HUN    Men   
1  1896  Athens  Aquatics   Swimming    HERSCHMANN, Otto     AUT    Men   
2  1896  Athens  Aquatics   Swimming   DRIVAS, Dimitrios     GRE    Men   
3  1896  Athens  Aquatics   Swimming  MALOKINIS, Ioannis     GRE    Men   
4  1896  Athens  Aquatics   Swimming  CHASAPIS, Spiridon     GRE    Men   

                        Event   Medal  
0              100M Freestyle    Gold  
1              100M Freestyle  Silver  
2  100M Freestyle For Sailors  Bronze  
3  100M Freestyle For Sailors    Gold  
4  100M Freestyle For Sailors  Silver

In [4]:
summer.tail()

Year    City      Sport           Discipline  \
31160  2012  London  Wrestling  Wrestling Freestyle   
31161  2012  London  Wrestling  Wrestling Freestyle   
31162  2012  London  Wrestling  Wrestling Freestyle   
31163  2012  London  Wrestling  Wrestling Freestyle   
31164  2012  London  Wrestling  Wrestling Freestyle   

                         Athlete Country Gender     Event   Medal  
31160         JANIKOWSKI, Damian     POL    Men  Wg 84 KG  Bronze  
31161  REZAEI, Ghasem Gholamreza     IRI    Men  Wg 96 KG    Gold  
31162             TOTROV, Rustam     RUS    Men  Wg 96 KG  Silver  
31163          ALEKSANYAN, Artur     ARM    Men  Wg 96 KG  Bronze  
31164             LIDBERG, Jimmy     SWE    Men  Wg 96 KG  Bronze

In [5]:
summer[summer.isna().any(axis=1)]


Year    City          Sport           Discipline          Athlete  \
29603  2012  London      Athletics            Athletics          Pending   
31072  2012  London  Weightlifting        Weightlifting          Pending   
31091  2012  London  Weightlifting        Weightlifting          Pending   
31110  2012  London      Wrestling  Wrestling Freestyle  KUDUKHOV, Besik   

      Country Gender     Event   Medal  
29603     NaN  Women     1500M    Gold  
31072     NaN  Women      63KG    Gold  
31091     NaN    Men      94KG  Silver  
31110     NaN    Men  Wf 60 KG  Silver

In [6]:
# Remover dados faltantes
summer.dropna(inplace=True)

In [7]:
# Alinhar as tabelas oficiais com os dados
wik_1976["Country"] = wik_1976.NOC.str.split("(", expand=True).iloc[:, 1].str.replace(")", "").str.replace("*", "")
wik_1976 = wik_1976.drop(columns=["Rank", "NOC", "Total"]).set_index("Country")
wik_1996["Country"] = wik_1996.Nation.str.split("(", expand=True).iloc[:, 1].str.replace(")", "").str.replace("*", "")
wik_1996 = wik_1996.drop(columns=["Rank", "Nation", "Total"]).set_index("Country")

# Passo 2: Criação da Coluna Event_Gender

In [8]:
# Identificar eventos mistos
mask1 = summer.Event.str.lower().str.contains("mixed")
mask2 = summer.Event.str.lower().str.contains("pairs")
mask3 = summer.Sport.str.lower().str.contains("equestrian")
mask4 = ((summer.Sport.str.lower().str.contains("sailing")) & (summer.Year < 1988))

summer.loc[mask1 | mask2 | mask3 | mask4, "Event_Gender"] = "X"

In [9]:
# Medals em Badminton misto
badm_mixed = [21773, 21782, 21776,21785, 21770, 21779,23703,23712,23706, 23715,23709,23700,25720,25729,25723,25732,25726,
              25717,27727,27736, 27730,27739,27724,27733, 29784, 29785,29786,29787,29788,29789]
summer.loc[badm_mixed, "Event_Gender"] = "X"

In [10]:
# Passo 3: Contagem de Medalhas por Evento
summer["Event_Medals"] = summer.groupby(["Year", "Sport", "Discipline", "Event", "Event_Gender"]).Medal.transform("count")

# Passo 4: Identificação de Eventos em Equipe

In [11]:
summer["Team"] = pd.Series(np.where(summer.Event_Medals > 5, "Yes", "No"))

# Passo 5: Remoção de Medalhas Duplicadas em Eventos de Equipe

In [12]:
summer.reset_index(inplace=True)
singles = summer.loc[summer.Team == "No"].copy()
team = summer.loc[summer.Team == "Yes"].copy()
team.drop_duplicates(subset=["Year", "Sport", "Discipline", "Country", "Event", "Event_Gender", "Medal"], inplace=True)
summer_new = pd.concat([singles, team])
summer_new.set_index("index", inplace=True)

# Passo 6: Criação da Tabela Oficial de Medalhas para Todas as Edições

In [13]:
medal_tables = summer_new.groupby(["Year", "Country", "Medal"]).Medal.count().unstack(fill_value=0)[["Gold", "Silver", "Bronze"]]

# Passo 7: Comparação com as Tabelas de Medalhas da Wikipedia

In [14]:
agg_1976 = medal_tables.loc[1976].sort_values(["Gold", "Silver", "Bronze"], ascending=False).copy()
div_76 = agg_1976.sub(wik_1976).abs().dropna()
score_76 = div_76.sum().sum()

In [15]:
agg_1996 = medal_tables.loc[1996].sort_values(["Gold", "Silver", "Bronze"], ascending=False).copy()
div_96 = agg_1996.sub(wik_1996).abs().dropna()
score_96 = div_96.sum().sum()

In [16]:
print(score_76, score_96)


650.0 987.0
